In [ ]:
import sys, os

In [ ]:
sys.path.insert(0, "../../python/rg17/")
import text_cleaning as tc

In [ ]:
from datawand.parametrization import ParamHelper
ph = ParamHelper("../../", "TrendApproximation", sys.argv)

# 1. Parameters

In [ ]:
experiment_id = ph.get("experiment_id")
experiment_prefix = ph.get("experiment_prefix")
output_dir = ph.get("experiment_dir")
flink_outs_prefix = ph.get("flink_outs_prefix")

In [ ]:
num_cols = 203 # keyword, date_from, date_to, list of words (100x2 with counts)
num_key_words = 100
kw_text_cols = [0] + [(i+1)*2+1 for i in range(num_key_words)]
kw_count_cols = [(i+1)*2+2 for i in range(num_key_words)]

In [ ]:
kw_text_cols[:4]

# 2. Load & Clean combined occurences

In [ ]:
file_path = "%s/%s.txt" % (flink_outs_prefix, experiment_id)

In [ ]:
occurences_pd = tc.get_occurences_data_frame(file_path, num_cols)

In [ ]:
len(occurences_pd)

## i.) Convert numerical columns

In [ ]:
occurences_pd[kw_count_cols] = occurences_pd[kw_count_cols].astype("float64")

## ii.) Introduce new columns

In [ ]:
occurences_pd["key_word"] = occurences_pd[0]
occurences_pd["count"] = occurences_pd[kw_count_cols].max(axis=1)
del occurences_pd[0]

In [ ]:
old_cols = list(occurences_pd.columns)
new_cols = ["start_time","end_time"] + list(range(1,201)) + ["key_word", "count"]
occurences_pd.columns = new_cols

## iii.) Changing column order

In [ ]:

occurences_pd = occurences_pd[["start_time","end_time"] + ["key_word", "count"] + list(range(1,201))]

# 3. Export occurences in tabular format

In [ ]:
occurences_pd.head(5)

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
output_file = "%s/occ_table.csv" % (output_dir)
print(output_file)
occurences_pd.to_csv(output_file,sep="|",index=False, header=True)